In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

In [2]:
r = requests.get("https://data.heatonresearch.com/data/t81-558/text/treasure_island.txt")
raw_text = r.text
print(raw_text[0:1000])

ï»¿The Project Gutenberg EBook of Treasure Island, by Robert Louis Stevenson

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: Treasure Island

Author: Robert Louis Stevenson

Illustrator: Milo Winter

Release Date: January 12, 2009 [EBook #27780]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK TREASURE ISLAND ***




Produced by Juliet Sutherland, Stephen Blundell and the
Online Distributed Proofreading Team at http://www.pgdp.net









 THE ILLUSTRATED CHILDREN'S LIBRARY


         _Treasure Island_

       Robert Louis Stevenson

          _Illustrated by_
            Milo Winter


           [Illustration]


           GRAMERCY BOOKS
              NEW YORK




 Foreword copyright Â© 1986 by Random House V


In [3]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text)

In [4]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 397400
total chars: 60


In [5]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 132454


In [6]:
sentences

['the project gutenberg ebook of treasure ',
 ' project gutenberg ebook of treasure isl',
 'oject gutenberg ebook of treasure island',
 'ct gutenberg ebook of treasure island, b',
 'gutenberg ebook of treasure island, by r',
 'enberg ebook of treasure island, by robe',
 'erg ebook of treasure island, by robert ',
 ' ebook of treasure island, by robert lou',
 'ook of treasure island, by robert louis ',
 ' of treasure island, by robert louis ste',
 ' treasure island, by robert louis steven',
 'easure island, by robert louis stevenson',
 'ure island, by robert louis stevenson\r\n\r',
 ' island, by robert louis stevenson\r\n\r\nth',
 'land, by robert louis stevenson\r\n\r\nthis ',
 'd, by robert louis stevenson\r\n\r\nthis ebo',
 'by robert louis stevenson\r\n\r\nthis ebook ',
 'robert louis stevenson\r\n\r\nthis ebook is ',
 'ert louis stevenson\r\n\r\nthis ebook is for',
 ' louis stevenson\r\n\r\nthis ebook is for th',
 'uis stevenson\r\n\r\nthis ebook is for the u',
 ' stevenson\r\n\r\n

In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [8]:
x.shape

(132454, 40, 60)

In [9]:
y.shape

(132454, 60)

In [10]:
y[0:10]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, Fal

In [11]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               96768     
_________________________________________________________________
dense (Dense)                (None, 60)                7740      
Total params: 104,508
Trainable params: 104,508
Non-trainable params: 0
_________________________________________________________________


In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Train on 132454 samples
Epoch 1/60
131840/132454 [============================>.] - ETA: 0s - loss: 2.0207****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "u can't; you
ain't got the invention of"
u can't; you
ain't got the invention of the stor the stor the stor the stor the stor and the stor and with a lood and the stor the captain. and the stor the stor and the stor the stor the stor silver in the stor the stor silver in the start of the still the stor the stor for the stor with the stor a sure and the stor the stor with the stored the stor the stor the stor silver he sea silver, and the stor the stor the stor the stor the st
----- temperature: 0.5
----- Generating with seed: "u can't; you
ain't got the invention of"
u can't; you
ain't got the invention of the sectire on the the story here and stare was a hims of his one still the stor was still a captain. you'll fer a

gutenberg-tm electronic works. the spare was for me of the eoder and to the stord and in the sudder was were another to the companion and so the hought been work and bring to the sat hands to the sea-chear of the sea-changer, and the slage in the sun the hands so the sound and all the bear of the sound of the strouthed and to the sime there in the shot and the first and been come to be
mean, who could you precuining the captai
----- temperature: 1.0
----- Generating with seed: "project
gutenberg-tm electronic works. "
project
gutenberg-tm electronic works. they anyth then clape, 'nnorking your ins another, paibed trem, and me--mmuder, and then wxlg slippen heaved captain spout pogcher were but wind had bady sead schier what'ing, and a
thought to the
verepe of join
the beck, wemparisthing for his mest after the stockadely begoachs bebongey--strong two-sw,w, you ain't here there!"

"i'll  aboard--a all land
to fange, moce overhenard i, i'm
and 
----- temperature: 1.2
----- Generating wit

shropy, captaged the wor? in pause 

ih were at ?ursc!, what i hadper's over side. you can a munn tove grippenrel, last beginny as i recount id_ plece on the guls was ringit-walking-nall, spifit, hatcupled jim is
now dody," said ill still elve laathed nar ender. now, "you nog on our ooth after musketst. i'll i ngremen in gran
raking
ejyune you fiarrled by my
ran dogtty, "that's the covigmver.
132454/132454 [==============================] - 65s 493us/sample - loss: 1.3881
Epoch 8/60
131968/132454 [============================>.] - ETA: 0s - loss: 1.3696****************************************************************************
----- Generating text after Epoch: 7
----- temperature: 0.2
----- Generating with seed: " bare-headed in the breeze. a good deal "
 bare-headed in the breeze. a good deal as i was the way of the woods of the woods of the water of the woods of the woods, and the woods of the pirate of the woods, and which we were the woods of the woods of the way of the water of 

was a ship when the same the sure the same for the same for the sea and the brought of the particular and complier of the strang
reast and with a sure the stranger had
already between the ship was a ship with a man of the sure the bottle founthing and shore.

"well," said the squire. "you are in the sure 
----- temperature: 0.5
----- Generating with seed: " the calf.
he was sure to recover, the "
 the calf.
he was sure to recover, the rest and set the staclly
short and fellow with the bulling in the brought had all the southing that i had a soul had
a man that for the stranger hands set for the rest and the captain sea
the addles, short.

"and you're not lay to the sure and thing in on the same for the whole of the current, who has the cabin was on the stranger, and a ships was every soul sail hard a coons that all was my 
----- temperature: 1.0
----- Generating with seed: " the calf.
he was sure to recover, the "
 the calf.
he was sure to recover, the partitions before in the first cr

we were heading ignan. and then i knly, guined to regries, mark lifted the doocion. then, current my ebove and everint black dog.

"and you've thought," he fear to forward.

there wen e'th no more very still jamble against
the. the recolor man down, but, patinnoven be uso it go
kept dopally about silver with asking, that recoinspering ake
the poge."

'em, are leng dedon't ting good the laws of that do
----- temperature: 1.2
----- Generating with seed: "ight the treasure island.
we were headi"
ight the treasure island.
we were heading i were pewyor, it. a
side, going for hisshrorhae's fire.

-flass, there. disking's silver." shoulding forrelts, and one pair as you
had on, un afney if you fannuon yipnter without enough, at it that had out
oncestien v'ilence, instep him are told boat. their persude.

overheasily, there continuly a. i like a
tling again, r. many coochotchs, the shaquon, sitch
ombank. iy overfulle. or .
132454/132454 [==============================] - 69s 521us/sample - los

...

aptain smollett, "i've a word to say to see it doubly was his ship. by the pieces of the sea, so the color in the first hand and brought and shipmed as the ship of the top of the
ship's in the project gutenberg-tm like on the ship
to have been a steep, and so the color in the door and sealed and and some still, i had soon as he saw in a musket was
for an agree of the growlons, and a donber in a great alow shoulder of the
rest had se
----- temperature: 1.0
----- Generating with seed: "aptain smollett, "i've a word to say to "
aptain smollett, "i've a word to say to ponty for the hseafe in. had not
made to and looknuened you would heg to and again, "had not ene-kinds on the south-'fhe's none ofingorocy. he wasning wondered.

mosteds; and, ratele more, and growled gring savery of the own,
again, in a powohs.

why, i began, countly about a way on alop's. i had out i over gave it. it were

well, i wall, told," said silver. his men, jim,
"then would like
----- temperature: 1.2
----- Generati

at the same time he beeded. this was shot like. 
places would have seen mo. vipumisis more in comidment appear the pen
from hims.

"killet's!" i head smollete iring. ahong that abouthance,
paved hill wated's, and violed mean given fone gone?"

"then, undir't
-lie"; whet on you'll there, than, e't our waitions along the
lapping, with his own owney. guid the pool,
my name, i'm otherwagns go strain, and be; and.
132454/132454 [==============================] - 182s 1ms/sample - loss: 1.2432
Epoch 25/60
131968/132454 [============================>.] - ETA: 0s - loss: 1.2395****************************************************************************
----- Generating text after Epoch: 24
----- temperature: 0.2
----- Generating with seed: " little to their taste; a sailor
is not"
 little to their taste; a sailor
is not head and the stockade, and the blue taken and a could of the sand of the stockade, and the same time and seen the stockade, and the same time
and the stockade, and the same tim

but we were all that had seen the boatsward of the boatsward of the
seas had a could on the party was the boatsward of the south, and the
sailing was the water stare and the black
of the particules, which i
----- temperature: 0.5
----- Generating with seed: " my man," interrupted the captain. "we k"
 my man," interrupted the captain. "we knowed visiting a such of the sailicale, and the black spot a pair as he was
to the ship was all been the others of the
captain sway. a till was silver before the three
hard or two and the
rest was a homent from my word, and summed the stern with the dead would be
sea. so my feet of the arms had said the
sea. i'll trund the cook the particulers.

"well, i had one sails was still trunks in

----- temperature: 1.0
----- Generating with seed: " my man," interrupted the captain. "we k"
 my man," interrupted the captain. "we kne will blow nited, and hearsented gind i was cablow never
sheer of the swipple fires, shi mate?"

"and i should would sbear bexinade

"fell to you stemnhaul silvor," he repeated.

if the marsmadly a moperhaine of not uath
into east to get it was all cracket i, but you puls jim we were lyied throuightn--and his spa
side us to distilision.  readanfright down and the younge like the tax ralu
----- temperature: 1.2
----- Generating with seed: "our to come frequent reports shook the i"
our to come frequent reports shook the iswar. ale'
lay's little little crue--apprehere histy up workeed gever, axgoatillost, from
look among as verientering, che, you remember his rore alll wenk
sion? i'll gret the wayoas upon a hond, and ," says me.

all silver's narnold; for it was neit held up shout with hawliling of
ame or sea sion of every you ho-wery buried
shot upon me allay, and then we were every nnot
yand of ame on her
132454/132454 [==============================] - 83s 625us/sample - loss: 1.2162
Epoch 32/60
131840/132454 [============================>.] - ETA: 0s - loss: 1.2118***************************************************

that men in the same time the ship that i saw my
delay before the sure it was so that down and the black dog, and the
sea. i seen hands and the ship to the ship to the
sea. i was stood that i saw the ship to the souther of the sea and a ship the ship to the souther of the southern of the ship the
sea of the ship to the same time the stockade,
silver in the same more for the stockade, and the doctor 
----- temperature: 0.5
----- Generating with seed: "n, is upside down. did you take
that me"
n, is upside down. did you take
that men in the same more up the chreed of a closk to leap to the southing
market, and a shoulder in the sand of the south. soon
a sound i was since the while of the unterdirum the time or tall the island i was the deck before the sea. they would he sone as i saw him over
the man that was soon a pipe, and what i saw in
spart of and jumbles to see the southle, and
i got the black spot in the ship tha
----- temperature: 1.0
----- Generating with seed: "n, is upside down

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())